# PART 1

## install package : beautifulsoup4  

In [1]:
pip install beautifulsoup4 

     |████████████████████████████████| 102kB 2.9MB/s ta 0:00:011
Note: you may need to restart the kernel to use updated packages.


## import lib

In [2]:
import urllib
import os,re
from bs4 import BeautifulSoup
import pandas as pd
pd.set_option('display.max_colwidth',200) 


## get raw table data

In [3]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(URL)
soup = BeautifulSoup(page)
page.close()



## filter raw data

In [4]:
tables = soup.findAll('table')
tab = tables[0]
#tab

## define the dataframe columns

In [5]:

column_names = ['PostalCode','Borough','Neighborhood'] 

df = pd.DataFrame(columns=column_names)
df

,PostalCode,Borough,Neighborhood


## Convert to dataframe

In [6]:
temp=[]
notmatchrow=0

for tr in tab.findAll('tr'):
    for td in tr.findAll('td'):
        temp.append(td.get_text())
    if len(temp) == 3:
       df = df.append({'PostalCode': temp[0],
                       'Borough': temp[1],
                       'Neighborhood': temp[2][:-1]}, ignore_index=True)
    else:
       notmatchrow=notmatchrow+1
    temp=[]

In [7]:
df.head()


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## verify all rows have been converted to dataframe

In [8]:

if df.shape[0]==len(tab.findAll('tr'))-notmatchrow:
    print('All rows have been converted to dataframe')
else:
    missingrow=len(tab.findAll('tr'))-notmatchrow-df.shape[0]
    print(missingrow , 'rows missed' )


All rows have been converted to dataframe


## Delete data of borough with null value

In [9]:
df2=df[df.Borough!='Not assigned']


## Sort data

In [10]:
df2.sort_values(by='PostalCode')
df2=df2.reset_index(drop='True')

In [11]:
df2

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
...,...,...,...
206,M8Z,Etobicoke,Kingsway Park South West
207,M8Z,Etobicoke,Mimico NW
208,M8Z,Etobicoke,The Queensway West
209,M8Z,Etobicoke,Royal York South West


## define the new dataframe columns

In [12]:

column_names = ['PostalCode','Borough','Neighborhood'] 

df_new = pd.DataFrame(columns=column_names)
df_new

,PostalCode,Borough,Neighborhood


## init data

In [13]:
pc_old=df2['PostalCode'][0]
df_merg_postalcode=df2['PostalCode'][0]
df_merg_borough=df2['Borough'][0]

df_merg_neighborhood=[]      


## Gen new dataframe with merged neighborhood

In [14]:
count=0
for pc in df2['PostalCode']:
    if pc==pc_old:
        if df2['Neighborhood'][count] == 'Not assigned':
            df_merg_neighborhood.append(df2['Borough'][count])
        else:
            df_merg_neighborhood.append(df2['Neighborhood'][count])
    else:
        tempstr=''
        for nb in df_merg_neighborhood:
            tempstr=tempstr + nb + ','   
        df_new = df_new.append({'PostalCode': df_merg_postalcode,
                                'Borough': df_merg_borough,
                                'Neighborhood': tempstr[:-1]}, ignore_index=True)  
        df_merg_postalcode=df2['PostalCode'][count]
        df_merg_borough=df2['Borough'][count]
        df_merg_neighborhood=[]
        if df2['Neighborhood'][count] == 'Not assigned':
            df_merg_neighborhood.append(df2['Borough'][count])
        else:
            df_merg_neighborhood.append(df2['Neighborhood'][count])
        pc_old=df2['PostalCode'][count]
   
    count+=1
    
tempstr=''
for nb in df_merg_neighborhood:
    tempstr=tempstr + nb + ','  
df_new = df_new.append({'PostalCode': df_merg_postalcode,
                        'Borough': df_merg_borough,
                        'Neighborhood': tempstr[:-1]}, ignore_index=True)  
        
    
        

In [15]:
df_new

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
...,...,...,...
98,M8X,Etobicoke,"The Kingsway,Montgomery Road,Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
101,M8Y,Etobicoke,"Humber Bay,King's Mill Park,Kingsway Park South East,Mimico NE,Old Mill South,The Queensway East,Royal York South East,Sunnylea"


In [16]:
df_new.shape

(103, 3)

# PART 2

## Read GEO data

In [17]:
df_geo=pd.read_csv('http://cocl.us/Geospatial_data')

In [18]:
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
df_new['Latitude']=''
df_new['Longitude']=''

In [20]:
for ix1,row1 in df_new.iterrows():
    for ix2,row2 in df_geo.iterrows():
        if df_new.iloc[ix1]['PostalCode']==df_geo.iloc[ix2]['Postal Code']:
            df_new.iloc[ix1]['Latitude']=df_geo.iloc[ix2]['Latitude']
            df_new.iloc[ix1]['Longitude']=df_geo.iloc[ix2]['Longitude']
            break

In [21]:
df_new

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7533,-79.3297
1,M4A,North York,Victoria Village,43.7259,-79.3156
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.6543,-79.3606
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.7185,-79.4648
4,M7A,Queen's Park,Queen's Park,43.6623,-79.3895
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway,Montgomery Road,Old Mill North",43.6537,-79.5069
99,M4Y,Downtown Toronto,Church and Wellesley,43.6659,-79.3832
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.6627,-79.3216
101,M8Y,Etobicoke,"Humber Bay,King's Mill Park,Kingsway Park South East,Mimico NE,Old Mill South,The Queensway East,Royal York South East,Sunnylea",43.6363,-79.4985


# PART 3

In [58]:

column_names = ['PostalCode','Borough','Neighborhood','Latitude','Longitude'] 

df_toronto = pd.DataFrame(columns=column_names)
df_toronto

,PostalCode,Borough,Neighborhood,Latitude,Longitude


In [59]:
for ix3,row3 in df_new.iterrows(): 
    if df_new.iloc[ix3]['Borough'].find('Toronto') != -1:
        df_toronto = df_toronto.append({'PostalCode':df_new.iloc[ix3]['PostalCode'],
                                      'Borough':df_new.iloc[ix3]['Borough'],
                                      'Neighborhood':df_new.iloc[ix3]['Neighborhood'],
                                      'Latitude':df_new.iloc[ix3]['Latitude'],
                                      'Longitude':df_new.iloc[ix3]['Longitude']
                                     }, ignore_index=True)

In [60]:
df_toronto

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,"Adelaide,King,Richmond",43.650571,-79.384568
8,M6H,West Toronto,"Dovercourt Village,Dufferin",43.669005,-79.442259
9,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",43.640816,-79.381752


In [64]:
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
        

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         237 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0        conda-forge
    geopy:         1.20.0-py_0      conda-forge

The following pac

### Get Toronto GEO info

In [65]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [68]:
# create map of Toronto latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)
#map_toronto

In [69]:
# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    

In [70]:
map_toronto